# Imports

## Preparatory Imports

In [1]:
import sys
import os

src_path = "../src"
sys.path.insert(0, src_path)

hf_token = os.environ["HUGGINGFACE_TOKEN"]

In [19]:
from datetime import datetime
import time

## Main Imports

In [113]:
from data import load_data
from client import Client
from tester import Tester
import pickle

In [204]:
with open("../data/runs/run2.pickle", "rb") as conversation_file:
    conversations = pickle.load(conversation_file)

# Test the Prototype

In [207]:
data = [load_data(f"../data/processed/train{i}.json") for i in range(1, 4)]
entries = data[0] | data[1]
c = Client(
    "mistralai/Mistral-Nemo-Instruct-2407",
    token=hf_token,
)
t = Tester(c, entries)
t.conversations = conversations
indices = sorted(list(entries.keys()))

In [209]:
N = len(t.conversations)
print(N)

1743


In [211]:
indices[N]

1755

In [213]:
print(t.conversations[indices[N-1]])
print(t.conversations[indices[N]])

KeyError: 1755

In [215]:
i = 0

In [ ]:
bad = []
while True:
    try:
        t.run(indices[N:])
    except Exception as e:
        print()
        N = len(t.conversations)-1
        if len(entries[indices[N]].answers) != len(t.conversations[indices[N]].answers):
            print(f"removing {indices[N]}")
            del t.conversations[indices[N]]
        with open(f"../data/runs/run2_{i}.pickle", "wb") as out:
            pickle.dump(t.conversations, out)
        i += 1
        if i == 100:
            break
        if "429 Client Error" in str(e):
            while datetime.now().minute != 1:
                time.sleep(60)
        else:
            bad.append(N)
            N += 1

In [ ]:
t.run(indices[N:])

In [174]:
sorted(list(t.conversations))[-5:]

[1751, 1752, 1753, 1754, 1756]

In [188]:
N = 1755

In [190]:
print(entries[N].answers)
print(t.conversations[N].answers)

['add(41.1, 27.3)', 'add(ANS0, 20.9)', 'add(ANS1, 17.0)', '12.0', 'add(ANS2, 12.0)']


KeyError: 1755

In [192]:
print(entries[N-1].answers)
print(t.conversations[N-1].answers)

['subtract(71.34, 54.12)', '54.12', 'divide(ANS0, 54.12)']
['subtract(71.34, 54.12)', '54.12', 'divide(ANS0, ANS1)']


In [182]:
del t.conversations[N]

In [194]:
for i in range(N):
    if i not in t.conversations.keys():
        print(i)

407
672
864
901
1030
1207
1274
1485
1588
1643
1670
1749


In [200]:
len(t.conversations)

1743

In [202]:
with open("../data/runs/run2.pickle", "wb") as out:
    pickle.dump(t.conversations, out)